# This codebook is used to invoke colab utils to annotate images for bounding boxes and to save the annotations in a json format thereafter.

In [1]:
!pip install -U --pre tensorflow=="2.2.0"

     |████████████████████████████████| 516.2MB 32kB/s 
     |████████████████████████████████| 3.0MB 43.4MB/s 
     |████████████████████████████████| 460kB 41.7MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [2]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2608, done.
remote: Counting objects: 100% (2608/2608), done.
remote: Compressing objects: 100% (2161/2161), done.
remote: Total 2608 (delta 651), reused 1266 (delta 413), pack-reused 0
Receiving objects: 100% (2608/2608), 32.57 MiB | 30.68 MiB/s, done.
Resolving deltas: 100% (651/651), done.


In [3]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1647891 sha256=44477119c6fe767780fe513699b5d7baa230a4aad9375525beb30218ab102ba9
  Stored in directory: /tmp/pip-ephem-wheel-cache-a33vyr1p/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.2-cp37-none-any.whl size=44011 sha256=eed4e57dbcabfeaa6790ae82a06dd9b5c235ac4c7288711e8802da6abadf898b
  Stored in directory: /root/.cache/pip/wheels/ee/ee/18/c466221ca6900e3efce2f4ea9c329288808679aecdcb2838d3
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=056a15b967aa286727cba7eec6b31877cb603a9071d476e5de914d894b93b80f
  Stored in directory: /root/.cache/pip/wheels/59/b1/91/f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=303ea1dd0460689629dfaa56f61fe48862f8201327132799a8a1

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.29.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.


In [7]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

In [8]:
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

In [9]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


# Step 1: Bounding Box Detection Detection on Standard Images

In [10]:
# Load the images as numpy array
train_image_dir = 'models/research/object_detection/test_images/'
names=os.listdir(train_image_dir)
train_images_np = []
for i in range(1, 4):
  image_path = os.path.join(train_image_dir, 'image' + str(i) + '.jpg')
  train_images_np.append(load_image_into_numpy_array(image_path))



In [11]:
gt_boxes = []
colab_utils.annotate(train_images_np, box_storage_pointer=gt_boxes)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'--boxes array populated--'

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'--boxes array populated--'

<IPython.core.display.Javascript object>

In [12]:
gt_boxes

[array([[0.055     , 0.03027344, 0.055     , 0.03027344],
        [0.04166667, 0.01953125, 0.85833333, 0.30175781],
        [0.10166667, 0.3984375 , 0.93833333, 0.97558594]]),
 array([[0.07666667, 0.43565089, 0.17333333, 0.4933432 ],
        [0.24666667, 0.20340237, 0.315     , 0.22559172],
        [0.37      , 0.34097633, 0.40666667, 0.36464497],
        [0.38      , 0.42233728, 0.425     , 0.44674556],
        [0.42166667, 0.79215976, 0.47      , 0.8158284 ],
        [0.55333333, 0.3816568 , 0.605     , 0.41198225],
        [0.675     , 0.07914201, 0.85166667, 0.12278107],
        [0.75666667, 0.15236686, 0.94666667, 0.20340237],
        [0.55833333, 0.05695266, 0.63166667, 0.08136095]]),
 array([[0.505     , 0.44427083, 0.63333333, 0.5390625 ],
        [0.46      , 0.55052083, 0.58166667, 0.62604167],
        [0.39666667, 0.7390625 , 0.54833333, 0.784375  ],
        [0.47166667, 0.83125   , 0.64      , 0.89322917],
        [0.415     , 0.7296875 , 0.55333333, 0.8265625 ]])]

In [13]:
#Connect the code to Google drive to dump the annotations there
# Now point to the folder with the images to be annotated
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import os
# The path below should point to the directory containing this notebook and the associated utility files
# Change it if necessary
os.chdir('/content/drive/MyDrive/Annotation/')
!ls

data_others.gdoc  data_others.txt  STARE


In [16]:
# Finally write the annotations to a file
import json
import numpy as np
data={}
data['bounding box']=[]
data['bounding box'].append(gt_boxes)

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

dumped = json.dumps(data, cls=NumpyEncoder)

In [17]:
with open('data_others.txt', 'w') as f:
    json.dump(dumped, f)

# Step 2: Annotate Medical Images from Google Drive

In [18]:
os.chdir('/content/drive/MyDrive/Annotation/STARE/jpgs/')
!ls

im0001.jpg  im0002.jpg	im0139.jpg


In [19]:
#Load the images as numpy array
train_image_dir = './'
names=os.listdir(train_image_dir)
train_images_np = []
for i in range(len(names)):
  image_path = os.path.join(train_image_dir, names[i])
  train_images_np.append(load_image_into_numpy_array(image_path))

In [20]:
gt_boxes = []
colab_utils.annotate(train_images_np, box_storage_pointer=gt_boxes)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'--boxes array populated--'

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'--boxes array populated--'

<IPython.core.display.Javascript object>

In [21]:
data={}
data['bounding box']=[]
data['bounding box'].append(gt_boxes)

In [22]:
with open('STARE_Bbos.txt', 'w') as f:
    json.dump(dumped, f)